*Notebook autogenerated from gpt2_sentiment_analysis.py on 08-Dec-2022*

*Notebook autogenerated from gpt2_sentiment_analysis.py on 07-Dec-2022*

# Sentiment analysis with GPT2 using IPUs


The Generative Pre-trained Transformer 2 (GPT-2) model can be utilised for text generation, question answering, translation and summarisation. 

In this notebook, we will show you how to quickly utilise Pytorch and the IPU for fast sentiment analysis using our implementation of GPT-2 small and medium on an IPU-POD4, and GPT-2 Large on an IPU-POD16.

First let's install our requirements and build our custom ops:

In [ ]:
# !pip install -r requirements.txt
# !make

# %load_ext autoreload
# %autoreload 2

import os
number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 16))

## Text Generation using GPT-2 Small

GPT-2 Small is a 124M parameter model pre-trained on a vast corpus of English data.

Our model checkpoints are collected from the Hugging Face model repository, you can learn more about the model checkpoint and the dataset used to train it on this [Hugging Face model card](https://huggingface.co/gpt2).

Let's see how we can use the generative capabilities of GPT-2 to help us decide what to cook for dinner tonight with the following prompt:

In [ ]:
generation_prompt = "Dinner is really important to me because..."

GPT-2 Small has a fewer number of parameters meaning that it can be run on a single IPU. 
The `Pipeline` class has default options which are specific to work on this system configuration, by setting 'gpt2' we are able to load and prepare the GPT-2 Small model to run inference on the IPU.

The model pipeline allows us to load and initialise the model on the IPU ready to quickly send the inputs from the host and receive the models outputs. 

In [ ]:
from text_inference_pipeline import Pipeline
hugging_face_gpt2_small = 'gpt2'
gen_pipeline = Pipeline('gpt2', prompt=generation_prompt)
print(gen_pipeline.output)

Since GPT-2 has been trained to be able to complete a wide range of tasks, we can reuse this pipeline to ask GPT-2 some more direct questions to see what it comes up with. 
Let's see GPT-2's ability to answer questions.

In [ ]:
print(gen_pipeline("Question: What should we cook for dinner tonight? Answer:"))

Few-Shot Learning is a method of feeding a model a very small training set to teach it to complete a specific task.
This small dataset offers the model 'context' to predict the next sequence of tokens more reliably as they are used as additional training samples.

What if we give GPT-2 some context by providing it with a finished question and answer, follow by asking another question? 

In [ ]:
question_answer = "Question: Why do we eat breakfast? Answer: Because its the most important meal of the day. ### "
question = "Question: What do you think? Answer:"

print(gen_pipeline(prompt = question, context = question_answer))

From doing this small exercise, we have been able to see how quickly we can utilise GPT-2 to help us generate text!
The generated text is quite random and without context the model was unable to correctly identify the task which we are trying to complete. 

On the other hand, by providing the model with context in the form of a question and answer, we can see that the output from the model becomes more reliable.

Next we will see that the GPT-2 model offers a large amount of flexibility by being able to quickly change tasks to Sentiment Analysis.
In the next section you will see how to use Few-Shot Learning to do Sentiment Analysis using the same initialised pipeline.

Since we have finished experimenting with this model we must also detach from the IPU using model.detachFromDevice() to free up resources:

In [ ]:
gen_pipeline.model.detachFromDevice()

## Sentiment analysis using Few-Shot learning

Using Few-Shot learning, we can guide GPT-2 to complete a more specific task by feeding the model context prior to the the input which we want the model to predict on.
We have briefly seen the benefit of doing this in the last section, now lets adapt the task to Sentiment Analysis!

Our sentence structure must have the following format for both our context prompt and our own inputs: "Message: `text` Sentiment: `classification` ###".

The `text` will be a sentence with a `positive` , `negative` or `neutral` classification, the sentence finishes with `###` as the stop token.

Using this structure, we will feed the model with the context it needs to predict the next token, let's use the following sentences for context:

In [ ]:
set1 = ["Message: The moon rotates around the earth. Sentiment: Neutral ### ",
        "Message: The weather has been horrible this winter… Sentiment: Negative ### "
        "Message: I love the IPU, it is so fast! Sentiment: Positive ### "
        ]
set2 = ["Message: I really like peanut butter on toast. Sentiment: Positive ### "
        "Message: The capital of Spain is Madrid. Sentiment: Neutral ### ",
        "Message: I have really bad luck. Sentiment: Negative ### "]

set3 = ["Message: I'm very excited for the party tomorrow night! Sentiment: Positive ### "
        "Message: My family are coming to my house for dinner. Sentiment: Neutral ###"
        "Message: Unfortunately, the train was delayed again. Sentiment: Negative ### "]

context_prompt = "".join(set1 + set2 + set3)

By concatenating these sentences together we have created the context needed to complete Few-Shot Learning on our model.

In [ ]:
print(context_prompt)

Finally, we can now include a test prompt which we want to use to see if our model correctly predicts the next token in the sequence.

In [ ]:
test_example = "Message: That was the best movie I've seen this year! Sentiment:"

`context prompt` will be used for additional experiments later on, hence we will keep this separate from the `test_example` prompt which we will use to test our model for Sentiment analysis.

### Using our GPT-2 Small Pipeline for Sentiment Analysis

Now, with our `context_prompt` and our `test_example` we can utilise the pipeline for fast sentiment analysis on the IPU!
First we must reinitialise the small_pipeline with the Hugging Face model checkpoint but with `topk=1` set in this instance. This is because for sentiment analysis we only care about the most likely classification.

In [ ]:
small_pipeline = Pipeline(hugging_face_gpt2_small, prompt=context_prompt + test_example, topk=1)

print(small_pipeline(test_example, context_prompt))

As we were hoping, our test token has been correctly classified to have a `Positive` sentiment.

Now that we have successfully used our pipelined model for sentiment analysis on our test prompt, we can continue to test our GPT-2 pipeline on experiments

In [ ]:
positive_test = "### Message: I'm excited for my trip to London this weekend! Sentiment:"
print(small_pipeline(prompt = positive_test, context = context_prompt))

negative_test = "### Message: I'm sad that it's raining today. Sentiment:"
print(small_pipeline(prompt = negative_test, context = context_prompt))

neutral_test = "### Message: I had a banana for breakfast this morning. Sentiment:"
print(small_pipeline(prompt = neutral_test, context = context_prompt))

Feel free to try to experiment with the small pipeline for your own prompts! 

Just remember to keep to the same format that we discussed earlier.

Use this code block to quickly experiment with your own prompts by swapping out the `user_input` string:

In [ ]:
user_input = "It's my turn!"
small_output = small_pipeline(prompt = "### Message: " + user_input + " Sentiment:", context = context_prompt)
print(small_output)

The outcomes of these tests are not quite accurate.
Since we are using the smallest version of GPT-2, there is a risk that the models predictive capabilities are not as accurate as we may require for this task.
By scaling up to a larger version of GPT-2, we should be able to have more accurate results.

Since we have finished experimenting with this model we must also detach from the IPU using model.detachFromDevice() to free up resources:

In [ ]:
small_pipeline.model.detachFromDevice()

### Using our GPT-2 Medium Pipeline for Sentiment Analysis

GPT-2 Medium has 355M parameters, and is the next size up from the GPT-2 model.
This implementation has more decoder layers fitted to the model, which allows us to achieve more accurate results. 

Due to this increase in parameters, we must pipeline our model across 4 IPUs and set specific configuration options related to this larger model.
To learn more about pipelining models on the IPU, see our [tutorial on this topic](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/pipelining).

We must set the following arguments in our run GPT2 pipeline on 4 IPUs:
 - `model_name_or_path` must be changed to 'gpt2-medium' to load the medium [Hugging Face checkpoint](https://huggingface.co/gpt2-medium),
 - `single_ipu` must now be set to `False`,
 - `layers_per_ipu` - specifies which of the model layers should be pipelined across the IPU, GPT-2 Medium is a 24 layer model which is split up across an IPU-POD4 with [1, 7, 8, 8] layers being placed on each chip respectively. 
 - `matmul_proportion` - allows us to control how much temporary memory is used when doing matrix multiplication and convolution operations on each chip, to learn more read our docs on [Available Memory Proportion](https://docs.graphcore.ai/projects/available-memory/en/latest/index.html). Since we are running on a POD4 we can the memory proportion for each chip as shown below.

In [ ]:
hugging_face_gpt2_medium = 'gpt2-medium'
medium_pipeline = Pipeline(hugging_face_gpt2_medium,
                           prompt = context_prompt + test_example,
                           single_ipu = False,
                           layers_per_ipu = [1, 7, 8, 8],
                           matmul_proportion = [0.2, 0.2, 0.2, 0.2],
                           topk=1)

Using the medium_pipeline and we should see that the model is able to preform better on Sentiment Analysis than the previous model.
Lets see if our new model is good at deciphering our test prompt using our new configuration:

In [ ]:
print(medium_pipeline(test_example, context_prompt))

Now we can test it against our additional experiments above. 
Again, feel free to experiment with your own prompts! 

In [ ]:
print(medium_pipeline(prompt = positive_test, context = context_prompt))

print(medium_pipeline(prompt = negative_test, context = context_prompt))

print(medium_pipeline(prompt = neutral_test, context = context_prompt))

You can now use this code block to quickly experiment with your own prompts on GPT-2 Medium:

In [ ]:
user_input = "It's my turn!"
medium_output = small_pipeline(prompt="### Message: " + user_input + " Sentiment:", context=context_prompt)
print(medium_output)

As we have seen by scaling up we were able to produce even more consistent results from our model.

Next, we can see how we can improve the model performance even more by running a larger model with a higher number of parameters.

Since we are done running experiments on this model, we must detach from the IPU again to free up resources like we did earlier, but this time on the `medium_pipeline`.

In [ ]:
medium_pipeline.model.detachFromDevice()

### Using our GPT-2 Large Pipeline for Sentiment Analysis

GPT 2 Large is a 36 layer model which has 774M parameters, and should provide you with even better results than the previous two models. 
If you have access to an IPU-POD16, you can push the abilities of GPT2 further by running inference our implementation of GPT2-Large.

Since this is a much larger implementation, we must pipeline the 36 layers in the model across 16 IPUs in order to fit them within the memory constraints of the IPU, as well as tuning the `matmul_proportion` constraint.
The arguments below allow us to reset these parameters to fit this new configuration.

In [ ]:
assert number_of_ipus in {16}, "Only IPU-POD16's are supported in this section of the notebook"

Let's get started by loading up the [GPT-2 Large](https://huggingface.co/gpt2-large) checkpoint from Hugging Face.

In [ ]:
hugging_face_gpt2_large = 'gpt2-large'
large_pipeline = Pipeline(hugging_face_gpt2_large,
                          prompt = generation_prompt,
                          single_ipu = False,
                          layers_per_ipu = [0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2],
                          matmul_proportion = [0.2, 0.15, 0.2, 0.2, 0.2, 0.15, 0.15, 0.2, 0.2, 0.15, 0.2, 0.2, 0.2, 0.15, 0.15, 0.2],
                          topk=1)

Lets see what happens if we generate text from the large pipeline.

In [ ]:
print(large_pipeline.output)

Now lets see what happens if we run the large pipeline on our sentiment analysis task, first lets have a look at our test example:

In [ ]:
print(large_pipeline(test_example, context_prompt))

print(large_pipeline(positive_test, context=context_prompt))
print(large_pipeline(negative_test, context=context_prompt))
print(large_pipeline(neutral_test, context=context_prompt))

As we have seen GPT-2 Large 

In [ ]:
len(test_example + context_prompt)

Finally, we can experiment with the large model!

In [ ]:
user_input = "It's my turn!"
large_output = large_pipeline("### Message: " + user_input + " Sentiment:", context=context_prompt)
print(large_output)

Now we can see that we have seen the incredible predictive capabilities of GPT-2 Large, we can detach from our device and conclude this demo.

In [ ]:
large_pipeline.model.detachFromDevice()

# Conclusion

This notebook has shown how quickly you can utilise the generative power of GPT-2 using IPU's!
We have seen how to quickly change tasks and scale up our model by swapping out different system configurations as we do that, we have also seen the positive impact of using larger models.